## Credit Card Fraud Detection

Autora: Jéssica Ramos

Dataset: https://www.kaggle.com/mlg-ulb/creditcardfraud

### Entendendo a base

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# leitura do dataset
data = pd.read_csv('./data/creditcard.csv')
data.head()

**Colunas do dataset:**

- Time: Segundos entre a primeira transação do dataset e a transação em questão.
- V1 a V28: Componentes principais obtidos das variáveis originais.
- Amount: Valor da transação.
- Class: A classificação da transação, sendo 1 = fraude e 0 = não fraude.

In [ ]:
# medidas descritivas
data.describe()

Todas as variáveis estão completas. 0.173% da base é composta de transações fraudulentas.

**Objetivo:** Criar um modelo de predição que classifique as transações como fraude ou não fraude.

#### Distribuição das variáveis nos grupos

In [ ]:
def boxplot(variable):
    sns.catplot(data = data, x = 'Class', y = variable, kind = 'box').set(title = variable)

In [ ]:
boxplot('Amount')

A variável `Amount` é bastante assimétrica à direita. Vou aplicar uma transformação de log para ser mais fácil visualizar a distribuição. Como existe o valor 0 em amount, vou também adicionar 1 unidade no valor da variável.

In [ ]:
data['log_Amount'] = data['Amount'].apply(lambda x: math.log(x+1))

boxplot('log_Amount')

As transações fraudulentas têm valores mais dispersos do que as transações legítimas.

As demais variáveis não têm interpertação direta associada, mas vou plotar os boxplots para ter uma ideia de separação entre os grupos.

In [ ]:
boxplot('V1')

In [ ]:
boxplot('V2')

In [ ]:
boxplot('V3')

In [ ]:
boxplot('V4')

In [ ]:
boxplot('V5')

In [ ]:
boxplot('V6')

In [ ]:
boxplot('V7')

In [ ]:
boxplot('V8')

In [ ]:
boxplot('V9')

In [ ]:
boxplot('V10')

In [ ]:
boxplot('V11')

In [ ]:
boxplot('V12')

In [ ]:
boxplot('V13')

In [ ]:
boxplot('V14')

In [ ]:
boxplot('V15')

In [ ]:
boxplot('V16')

In [ ]:
boxplot('V17')

In [ ]:
boxplot('V18')

In [ ]:
boxplot('V19')

In [ ]:
boxplot('V20')

In [ ]:
boxplot('V21')

In [ ]:
boxplot('V22')

In [ ]:
boxplot('V23')

In [ ]:
boxplot('V24')

In [ ]:
boxplot('V25')

In [ ]:
boxplot('V26')

In [ ]:
boxplot('V27')

In [ ]:
boxplot('V28')

Não existe clara separação na maioria das distribuições. Em geral, a classe de transações fraudulentas tem valores mais dispersos. Ambas as classes têm distribuições com muitos outliers.

Como as variáveis são resultado de PCA, não precisamos nos preocupar com colinearidades entre elas.

### Separação das bases de treino e teste

Vou dividir a base usando 70% para treino e 30% para teste. As transações da base de teste serão as últimas, usando a variável Time como referência.

In [ ]:
# calcula o percentil
perc70 = data[['Time']].quantile(0.7)
perc70[0]

In [ ]:
# cria a base de treino
X_train = data[data['Time'] <= perc70[0]].copy()
X_train.drop(['Class','Time','Amount'], axis = 1, inplace = True)
y_train = data[data['Time'] <= perc70[0]]['Class'].copy()

# cria a base de teste
X_test = data[data['Time'] > perc70[0]].copy()
X_test.drop(['Class','Time','Amount'], axis = 1, inplace = True)
y_test = data[data['Time'] > perc70[0]]['Class'].copy()

In [ ]:
# preditores treino
X_train.shape

In [ ]:
# preditores teste
X_test.shape

In [ ]:
# distribução do target na base de treino
y_train.describe()

In [ ]:
# distribução do target na base de teste
y_test.describe()

A base de treino tem 199.368 observações, sendo 0.193% fraudes. A base de teste tem 85.439 observações, sendo 0.126% fraudes.

### Corrigindo o desbalanceamento

A base de treino é muito desbalanceada, o que pode fazer os resultados não serem muito bons para a classe positiva.Para tentar corrigir o desbalanceamento, vou aplicar SMOTE apenas na classe positiva.

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(sampling_strategy = 'minority') # apenas afeta a classe positiva
X_train_new, y_train_new = oversample.fit_resample(X_train, y_train)

In [ ]:
# resultado
y_train_new.describe()

A nova base gerada tem 397.968 observações, sendo 50% transações fraudulentas.

### Modelo Logístico

Primeiro, vou ajustar um modelo logístico tradicional. As métricas de referência serão recall, specificity (recall da categoria negativa) e precision.

Escolhi o recall como métrica principal pensando num problema de fraudes em que transações com alto risco de fraude passariam por uma segunda autenticação. Portanto, é importante que as transações de fato fraudulentas tenham risco alto, ainda que existam vários falsos positivos.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression

In [ ]:
# fit do modelo logístico
lr = LogisticRegression()
lr.fit(X_train_new, y_train_new)

# predições
y_pred_train_lr = lr.predict(X_train_new)
y_pred_test_lr = lr.predict(X_test)

In [ ]:
# métricas de treino
metric_train_lr = precision_recall_fscore_support(y_train_new, y_pred_train_lr)

print('Precision: ', metric_train_lr[0][1],
      '\nRecall: ', metric_train_lr[1][1],
      '\nSpecificity: ', metric_train_lr[1][0])

In [ ]:
# métricas de teste
metric_test_lr = precision_recall_fscore_support(y_test, y_pred_test_lr)

print('Precision: ', metric_test_lr[0][1],
      '\nRecall: ', metric_test_lr[1][1],
      '\nSpecificity: ', metric_test_lr[1][0])

In [ ]:
# observações classificadas como positivas
y_pred_test_lr.mean()

O modelo logístico acertou 89.8% das observações fraudulentas e 97.4% das não fraudulentas na base de teste. Dentre as 2.7% de observações classificadas como fraude, 4.1% são de fato fraudes.

O modelo por default já considera regularização l2. Posso tentar otimizar o parâmetro usando cross validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

# define valores de C para testar
params_l2 = {'C': [10.0, 5.0, 2.0, 1.5, 1.0, 0.8, 0.5, 0.1, 0.01, 0.001]} # quanto menor, maior o peso do termo L2

l2_grid = GridSearchCV(estimator = LogisticRegression(),
                       param_grid = params_l2,
                       scoring = 'recall',
                       cv = 10,
                       verbose = 2)

# ajusta
l2_grid.fit(X_train_new, y_train_new)

In [ ]:
# resultado
pd.DataFrame(l2_grid.cv_results_).head(10)

A variação entre os resultados é de fato bem pequena. Portanto vou manter o modelo default que já ajustamos como o melhor resultado para a regressão logística.

### Random Forest

O modelo linear teve resultados bons, mas vou tentar melhorar o resultado com uma Random Forest. Existem vários parâmetros que podem ser otimizados, portanto usarei cross validation e uma random grid search para encontrar a melhor combinação.

Os valores na lista de busca são bem arbitrários, fui alterando à medida que rodei alguns resultados.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# define a grid de hiperparâmetos
params_rf = {'n_estimators': [100, 200, 300, 500, 800, 1000], # número de árvores
             'max_depth': [3, 4, 5, 6, 7, 8], # profundidade máxima de cada arvore
             'min_samples_leaf': [5, 6, 7, 10, 15, 20], # mínimo de amostras por folha
             'max_features': [0.7, 0.8, 0.9, 1.0]} # proporção das features consideradas em cada split

rf_grid = RandomizedSearchCV(estimator = RandomForestClassifier(),
                             param_distributions = params_rf,
                             scoring = 'recall',
                             n_iter = 5,
                             cv = 5,
                             random_state = 10,
                             verbose = 2)

# ajusta
rf_grid.fit(X_train_new, y_train_new)

In [ ]:
# resultado
pd.DataFrame(rf_grid.cv_results_).sort_values(by = 'rank_test_score').head(5)

In [ ]:
# melhor modelo
rf = rf_grid.best_estimator_

# predições
y_pred_train_rf = rf.predict(X_train_new)
y_pred_test_rf = rf.predict(X_test)

In [ ]:
# métricas de treino
metric_train_rf = precision_recall_fscore_support(y_train_new, y_pred_train_rf)

print('Precision: ', metric_train_rf[0][1],
      '\nRecall: ', metric_train_rf[1][1],
      '\nSpecificity: ', metric_train_rf[1][0])

In [ ]:
# métricas de teste
metric_test_rf = precision_recall_fscore_support(y_test, y_pred_test_rf)

print('Precision: ', metric_test_rf[0][1],
      '\nRecall: ', metric_test_rf[1][1],
      '\nSpecificity: ', metric_test_rf[1][0])

Comenta os resultados aqui

### Gradient Boosting

Vou avaliar também um Gradient Boosting. Também faço aqui a busca por hiperparâmetros ótimos.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# define a grid de hiperparâmetos
params_gb = {'learn_rate': [0.1, 0.05, 0.01, 0.001], # taxa de aprendizado
             'n_estimators': [100, 200, 300, 500, 800, 1000], # número de árvores
             'max_depth': [3, 4, 5, 6, 7, 8], # profundidade máxima de cada arvore
             'min_samples_leaf': [5, 6, 7, 10, 15, 20], # mínimo de amostras por folha
             'max_features': [0.7, 0.8, 0.9, 1.0]} # proporção das features consideradas em cada split

gb_grid = RandomizedSearchCV(estimator = GradientBoostingClassifier(),
                             param_distributions = params_rf,
                             scoring = 'recall',
                             n_iter = 5,
                             cv = 5,
                             random_state = 10,
                             verbose = 2)

# ajusta
gb_grid.fit(X_train_new, y_train_new)

In [ ]:
# resultado
pd.DataFrame(gb_grid.cv_results_).sort_values(by = 'rank_test_score').head(5)

In [ ]:
# melhor modelo
gb = gb_grid.best_estimator_

# predições
y_pred_train_gb = rf.predict(X_train_new)
y_pred_test_gb = rf.predict(X_test)

In [ ]:
# métricas de treino
metric_train_gb = precision_recall_fscore_support(y_train_new, y_pred_train_gb)

print('Precision: ', metric_train_gb[0][1],
      '\nRecall: ', metric_train_gb[1][1],
      '\nSpecificity: ', metric_train_gb[1][0])

In [ ]:
# métricas de teste
metric_test_gb = precision_recall_fscore_support(y_test, y_pred_test_gb)

print('Precision: ', metric_test_gb[0][1],
      '\nRecall: ', metric_test_gb[1][1],
      '\nSpecificity: ', metric_test_gb[1][0])

Comenta os resultados aqui

### Resultados finais do melhor modelo

Até então os modelos foram avaliados classificando como categoria 1 (fraude) as transações com probabilidade estimada > 0.5 e 0 caso contrário. Posso agora avaliar a distribuição da probabilidade estimada do melhor modelo.

In [ ]:
# dar uma analisada na distribuição aqui, definir outro corte se parecer melhor, exibir as métricas finais

In [ ]:
# analisar variáveis de maior importância pro modelo

In [ ]:
# concluir a análise